# Optimize/Deoptimize Polymorphism

In this notebook, we take a look at what happens to the function `is_even` below when invoking it many times, first with values of the same type and then with values of different type.

In [1]:
import s6

@s6.jit
def is_even(x):
    return x % 2 == 0

At this point, the function `is_even` has not been optimized by S6, as we haven't invoked it yet and there's no tracing information available. S6 needs to gather information about how it is used (e.g., what's the type of the parameter).

The property `is_compiled` tells us it isn't compiled.

In [2]:
print(s6.inspect(is_even).is_compiled)

False


Although the function isn't compiled yet, we time it for comparison later on.

In [3]:
%time is_even(10)

CPU times: user 347 µs, sys: 40 µs, total: 387 µs
Wall time: 393 µs


True

Once this function is used many times (becomes _hot_), and S6 has collected enough information about it, it will compile it to strongjit. `is_compiled` will then return `True` and we will be able to print the resulting `strongjit`.

In [4]:
for i in range(20000):
    is_even(i)
    
print(s6.inspect(is_even).is_compiled)
print(s6.inspect(is_even).strongjit)

True
type_feedback @4 monomorphic, int#9
type_feedback @8 monomorphic, int#9

function is_even {
&0: [ %1 ]                                                  // entry point
  bytecode_begin @0 fastlocals [%1]                         // LOAD_FAST 3839668643.py:5
  %3 = frame_variable consts, $1
  %4 = unbox long %1
  %5 = overflowed? %4
  deoptimize_if_safepoint %5, @4 stack [%1, %3] fastlocals [%1] increfs [%1, %3], "Value was not unboxable" // BINARY_MODULO 3839668643.py:5
  %7 = constant $2
  %8 = remainder i64 %4, %7
  %9 = overflowed? %8
  deoptimize_if_safepoint %9, @4 stack [%1, %3] fastlocals [%1] increfs [%1, %3], "Value was not unboxable" // BINARY_MODULO 3839668643.py:5
  %11 = constant $0
  %12 = cmp eq i64 %8, %11
  %13 = box bool %12
  advance_profile_counter $6
  decref notnull %1 @10                                     // RETURN_VALUE 3839668643.py:5
  return %13
}


Given that we always passed an integer as argument, S6 optimizes `is_even` for this particular type.

```
type_feedback @4 monomorphic, int#9
```
If we now time it, we get a much faster call:

In [5]:
%time is_even(5)

CPU times: user 17 µs, sys: 1 µs, total: 18 µs
Wall time: 21.9 µs


False

Although this function has been optimized for the integer case, it is still possible to invoke it with values of different types:

In [6]:
%time is_even(6.0)

CPU times: user 275 µs, sys: 21 µs, total: 296 µs
Wall time: 300 µs


True

It just runs much slower! Given that S6 considers the parameter of this function to be an integer (monomorphic!) and we call it with a float, S6 needs to _deoptimize_ it, i.e., going back to the interpreted version.

In [7]:
s6.inspect(is_even).is_compiled

False

S6 can compile it again, though. And in this case, the resulting code is prepared for both integers and floats:

In [8]:
s6.inspect(is_even).force_compile()

assert s6.inspect(is_even).is_compiled
print(s6.inspect(is_even).strongjit)


type_feedback @4 polymorphic, either int#9 or float#155
type_feedback @8 polymorphic, either int#9 or float#155

function is_even {
&0: [ %1 ]                                                  // entry point
  bytecode_begin @0 fastlocals [%1]                         // LOAD_FAST 3839668643.py:5
  %3 = frame_variable consts, $1
  %4 = call_native PyNumber_Remainder (%1, %3) @4           // BINARY_MODULO 3839668643.py:5
  %5 = constant $0
  %6 = cmp eq i64 %4, %5
  deoptimize_if %6, &22, &8, materializing values [%1]

&8:                                                         // preds: &0
  bytecode_begin @6 stack [%4] fastlocals [%1]              // LOAD_CONST 3839668643.py:5
  %10 = frame_variable consts, $2
  %11 = constant $2
  %12 = call_native PyObject_RichCompare (%4, %10, %11) @8  // COMPARE_OP 3839668643.py:5
  decref notnull %4 @8                                      // COMPARE_OP 3839668643.py:5
  %14 = constant $0
  %15 = cmp eq i64 %12, %14
  deoptimize_if %15, &25, &17, ma

The type feedbacks are now polymorphic (`type_feedback @4 polymorphic, either int#9 or float#155`) instead of monomorphic, and the generated code is also more complex than in the monomorphic version. S6 no longer tries to simply `unbox` the argument, or use `remainder i64` and `cmp eq i64` instructions. Instead, it calls the `PyNumber_Remainder` and `PyObject_RichCompare` functions.

Even though the code relies on these native calls, if we now time both function calls, we still get much better timings: 

In [9]:
%time is_even(6.0) 
%time is_even(6) 

CPU times: user 34 µs, sys: 2 µs, total: 36 µs
Wall time: 39.6 µs
CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 9.78 µs


True